# Emission files for Sven's test case

Produce point source emission and corresponding temporal and vertical profiles for Sven Werchner's simple test case with H2O2 chemistry

- produce point-source emission file for OEM
- produce vertical profile
- produce temporal profile

## Create Point source emission file
The file is created for the ICON grid provided by Sven with two point sources releasing H2O2

In [1]:
# imports
from pathlib import Path
from emiproc.inventories import Inventory
from emiproc.grids import ICONGrid, WGS84_PROJECTED

from emiproc.regrid import remap_inventory
from emiproc.utilities import SEC_PER_YR
from emiproc.exports.icon import export_icon_oem, get_constant_time_profile

from shapely.geometry import Point 
import geopandas as gpd
import numpy as np

/users/dbrunner/emiproc_venv/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/users/dbrunner/python/emiproc/emiproc/emiproc/inventories/__init__.py:10: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shap

ImportError: cannot import name 'get_constant_time_profile' from 'emiproc.exports.icon' (/users/dbrunner/python/emiproc/emiproc/emiproc/exports/icon.py)

In [ ]:
# create inventory object from a GeoDataFrame (gdf) with the two point sources
inv=Inventory.from_gdf(gdfs={
    'point_source1': gpd.GeoDataFrame(
        {
        "H2O2": [4*SEC_PER_YR],
        },
        geometry=[Point(8.4,49.0)],
        crs="EPSG:4326",
    ),
    'point_source2': gpd.GeoDataFrame(
        {
        "H2O2": [4*SEC_PER_YR],
        },
        geometry=[Point(9.2,48.8)],
        crs="EPSG:4326",
    ),
    }
)

# Convert to a planar crs for the remapping to work
inv.to_crs(WGS84_PROJECTED)

## Add the profiles

In [ ]:
# Load from csv
from emiproc.profiles.vertical_profiles import VerticalProfile

vp1 = VerticalProfile(
    ratios=np.array([0, 1]),
    height=np.array([499, 501])
)
vp2 = VerticalProfile(
    ratios=np.array([0, 1]),
    height=np.array([999, 1001])
)
# Set an empty vertical profile for H2O2
inv.set_profile(vp1, substance="H2O2", category="point_source1")
inv.set_profile(vp2, substance="H2O2", category="point_source2")
inv.set_profile(get_constant_time_profile(), substance="H2O2")


In [ ]:
# Load  icon grid and remap the inventory to this grid
grid_file = Path(
    r"path/to/grid.nc"
)
icon_grid = ICONGrid(grid_file)



In [ ]:
remapped_points = remap_inventory(inv, icon_grid)

In [ ]:
# export to ICON-compatible file
export_icon_oem(
    inv=remapped_points,
    icon_grid_file=grid_file,
    output_dir=grid_file.with_stem(f"{grid_file.stem}_with_h2o2_emissions"),
)

In [ ]:
# Investigate the output
import xarray as xr

ds = xr.load_dataset(r"path/to/outputs/oem_gridded_emissions.nc")

np.unique(ds['point_source2-H2O2'])